# Striatum example network

In this notebook we create and simulate an example striatum network. Since it is run locally we have reduced the number of neurons, so to run the full network you have to increase the neuron numbers listed below.

### Start ipcluster

Please first start ```ipcluster```, on linux you type the following into the terminal, note that you should be in the ```Snudda/examples/notebook``` directory:

```
export IPYTHONDIR="`pwd`/.ipython"
export IPYTHON_PROFILE=default
ipcluster start 
```

These commands are in ```start_ipcluster.sh``` in the ```examples/notebook``` directory. After starting ipcluster give it a few seconds to get fully up and running before proceeding with the rest of the commands in this notebook. 

If you restart the run you might need to do:

```
ipcluster stop
ipcluster start
```

For instructions how to install and activate ipyparallel for your notebook see 
https://github.com/ipython/ipyparallel

In [1]:
import os
from ipyparallel import Client

os.environ["IPYTHONDIR"] = ".ipython"
os.environ["IPYTHON_PROFILE"] = "default"

u_file = os.path.join(".ipython", "profile_default", "security", "ipcontroller-client.json")
if os.path.exists(u_file):
    rc = Client(url_file=u_file, timeout=120, debug=False)
else:
    print("Warning, running in serial.")
    rc = None

Warning, running in serial.


In ```struct_def``` we define the number of neurons to simulate in the striatum. Since this is an example we do a quite small network.

In [2]:
import os

network_path = os.path.join("networks/striatum_example")

from snudda import SnuddaInit
struct_def = {"Striatum": 20 }

si = SnuddaInit(network_path=network_path, struct_def=struct_def, random_seed=123)

Adding Striatum with 20 neurons
Using cube for striatum
Adding neurons: dSPN from dir $SNUDDA_DATA/neurons/striatum/dspn
Adding neurons: iSPN from dir $SNUDDA_DATA/neurons/striatum/ispn
Writing networks/striatum_example/network-config.json


In [3]:
from snudda import SnuddaPlace
sp = SnuddaPlace(network_path=network_path, rc=rc)
sp.place()

In [4]:
from snudda import SnuddaDetect

sd = SnuddaDetect(network_path=network_path, rc=rc)
sd.detect()

No d_view specified, running distribute neurons in serial
Processing hyper voxel : 1/64 (20 neurons)
Processing hyper voxel : 5/64 (20 neurons)
Processing hyper voxel : 21/64 (20 neurons)
Processing hyper voxel : 16/64 (20 neurons)
Processing hyper voxel : 17/64 (20 neurons)
Processing hyper voxel : 20/64 (19 neurons)
Processing hyper voxel : 4/64 (16 neurons)
Processing hyper voxel : 0/64 (14 neurons)
Processing hyper voxel : 22/64 (7 neurons)
Processing hyper voxel : 33/64 (5 neurons)
Processing hyper voxel : 18/64 (5 neurons)
Processing hyper voxel : 37/64 (5 neurons)
Processing hyper voxel : 6/64 (3 neurons)
Processing hyper voxel : 2/64 (3 neurons)
Processing hyper voxel : 25/64 (2 neurons)
Processing hyper voxel : 36/64 (2 neurons)
Processing hyper voxel : 9/64 (1 neurons)
Processing hyper voxel : 32/64 (1 neurons)
Processing hyper voxel : 38/64 (1 neurons)


In [5]:
from snudda import SnuddaPrune

sp = SnuddaPrune(network_path=network_path, rc=rc)
sp.prune()

Read 6915 out of total 6915 synapses


To generate input and simulate this network in NEURON please see ```striatum_example_simulate.ipynb```.


**Important** you need to shut down the kernel in this notebook before running the next notebook, otherwise the hdf5 file will complain that it is already locked.
